In [1]:
class  BaseModel():
    def __init__(self, model_name = ''):
        
        self.model =  ort.InferenceSession(model_name)
    def preprocess(self, input):
        return input
        
    def run(self, inputs):
        return self.model.run(output_names=[ "output" ], input_feed=dict(inputs))

    def postprocess(self, output):
        return output

In [2]:
import jellyfish

# Речь в текст

In [4]:
from optimum.onnxruntime import ORTModelForQuestionAnswering, ORTModelForAudioClassification, ORTModelForSequenceClassification, ORTModelForSpeechSeq2Seq
from transformers import AutoConfig, Wav2Vec2Processor, AutoTokenizer, PretrainedConfig, WhisperProcessor
import time
import librosa
import numpy as np
import os

class  SpeechtoText(BaseModel):
    def __init__(self, model_path, model_name, model_name_preprocess, postprocess = None):
        self.processor =  WhisperProcessor.from_pretrained(model_name_preprocess)#"openai/whisper-tiny")
        # self.sampling_rate = processor.feature_extractor.sampling_rate
        model_config = PretrainedConfig.from_pretrained(model_name)
        predictions = []
        references = []
        sessions = ORTModelForSpeechSeq2Seq.load_model(
            os.path.join(model_path, 'encoder_model.onnx'),
            os.path.join(model_path, 'decoder_model.onnx'),
            os.path.join(model_path, 'decoder_with_past_model.onnx'))
        self.model = ORTModelForSpeechSeq2Seq(sessions[0], sessions[1], model_config, model_path, sessions[2])
        self.forced_decoder_ids = self.processor.get_decoder_prompt_ids(language="russian", task="transcribe")
        if  postprocess is not None:
            self.post_process =  postprocess
        else:
            self.post_process = None
        self.sampling_rate = 16000
    def postprocess(self, output):

        output = self.processor.batch_decode(output, skip_special_tokens=True)
        return output
        
    def run(self, speech):
        feature = self.processor(speech, sampling_rate=self.sampling_rate, return_tensors="pt")
        rez = self.model.generate(feature['input_features'], forced_decoder_ids=self.forced_decoder_ids)
        # print(rez.logits[0], self.emotions)
        return self.postprocess(rez)

sampling_rate =  16000
path_a ='01_happiness_anger a_020.wav'
speech, sr = librosa.load(path_a, sr=sampling_rate)
model_text = SpeechtoText('whisper-tiny_onnx', "openai/whisper-base", "openai/whisper-tiny")
model_text.run(speech)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


[' Слушай, я потратил обретьом кучу денег для того, чтобы притоситься в эту дару. Это что вообще такое? Посмотри на официантов, они все в черных каких-то рубашках с кислыми минами. Даже никто из них до сих пор не подошел к нам.']

# Речь в эмоции

In [5]:

class  SpeechtoEmotion(BaseModel):
    def __init__(self, model_name, model_name_preprocess, model_postprocess = None):
        self.processor = Wav2Vec2Processor.from_pretrained(model_name_preprocess)
        self.sampling_rate = self.processor.feature_extractor.sampling_rate
        self.model = ORTModelForAudioClassification.from_pretrained(model_name)#"wav2vec2-xls-r-300m-emotion-ru_onnx"
        self.emotions = ['neutral', 'positive', 'angry', 'sad', 'other']
        if  model_postprocess is not None:
            self.post_process =  model_postprocess
        else:
            self.post_process = None
    
    def postprocess(self, outputs):
        return self.emotions[np.argmax(outputs.logits[0])]
        
    def run(self, speech):
        features = self.processor(speech, sampling_rate=self.sampling_rate, return_tensors="pt", padding=True)
        rez = self.model(features['input_values'])
        # print(rez.logits[0], self.emotions)
        return self.postprocess(rez)


path_a ='01_happiness_anger a_020.wav'
speech, sr = librosa.load(path_a, sr=sampling_rate)
model_em = SpeechtoEmotion("wav2vec2-xls-r-300m-emotion-ru_onnx", "KELONMYOSA/wav2vec2-xls-r-300m-emotion-ru")
model_em.run(speech)

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


/home/boss/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


'angry'

# Текст в эмоции

In [6]:
class  TexttoEmotion(BaseModel):
    def __init__(self, model_name, model_name_or_path):
        self.processor = AutoTokenizer.from_pretrained(model_name_or_path)
        self.model = ORTModelForSequenceClassification.from_pretrained(model_name)
        self.labels = ['neutral', 'joy', 'sadness', 'anger', 'enthusiasm', 'surprise', 'disgust', 'fear', 'guilt', 'shame']
        self.labels_ru = ['нейтрально', 'радость', 'грусть', 'гнев', 'интерес', 'удивление', 'отвращение', 'страх', 'вина', 'стыд']
    
    def postprocess(self, outputs):
        return self.labels[np.argmax(outputs.logits[0])]
        
    def run(self, text):
        features = self.processor(text,  max_length=512, truncation=True, return_tensors='pt')
        rez = self.model(**features)
        # print(features )
        return self.postprocess(rez)


model_textem = TexttoEmotion("rubert-tiny2-russian-emotion-detection_onnx", "Djacon/rubert-tiny2-russian-emotion-detection")
model_textem.run("ой беда, какая беда")

'sadness'

## Общий поток распознавания

In [7]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [16]:
def main_audio(model_text, model_textem, model_em, speech=None, dt=1, t=0):
    if speech is not None:
        speech_d = speech
        rez_text = model_text.run(speech)
        print(rez_text)
        rez_word = rez_text[0].split()[0]

        rez_em_text = model_textem.run(rez_text)
    
        
        rez_em_speech = model_em.run(speech)
        # запись результата : время, текст, слово для привязки времени, эмоция по тексту, Эмоция по звуку
        rez_ = [t * float(dt) / 2, rez_text[0], rez_word , rez_em_text, rez_em_speech]
        # print('Time: ', t1-t0)
        # print('Result text: ',rez_text)
        # print('Result emotion: ', rez_em_text, rez_em_speech)
        return rez_
    else:
        return [t * float(dt) / 2, '', '' , None, None]

In [17]:
path_a ='01_happiness_anger a_020.wav'
# path_a = '1725.wav'

speech, sr = librosa.load(path_a, sr=sampling_rate)
# создаем модели
model_text = SpeechtoText('whisper-tiny_onnx', "openai/whisper-base", "openai/whisper-tiny")
model_textem = TexttoEmotion("rubert-tiny2-russian-emotion-detection_onnx", "Djacon/rubert-tiny2-russian-emotion-detection")
model_em = SpeechtoEmotion("wav2vec2-xls-r-300m-emotion-ru_onnx", "KELONMYOSA/wav2vec2-xls-r-300m-emotion-ru")
# длина эпизода записи в с. 
dt = 1
d = 16000 * dt
n_d = 4
rez_text_all = model_text.run(speech)
rez_text_all_buf = rez_text_all[0] 
rez_text_all_list = rez_text_all[0].split(' ')
rez_all_result = []
l_speech = int(len(rez_text_all_buf) / (len(speech)/d * n_d) )
# распознавание
t0 = time.time()
for t, speech_i in enumerate(range(0,speech.shape[0], d)):
    t1 = time.time()
    speech_d = speech[speech_i:speech_i+d * n_d]
    rez_ = main_audio(model_text, model_textem, model_em, speech=speech_d, dt=1, t=0)
    # запись результата : время, текст, слово для привязки времени, эмоция по тексту, Эмоция по звуку
    rez_all_result.append( rez_)    
    print('Time: ', t1-t0)
    print('Result text: ',rez_text)
    print('Result emotion: ', rez_em_text, rez_em_speech)




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
[' Слушай, потратил об этом кучу денег. Для того, чтобы предощиться.']
Time:  6.67572021484375e-05
Result text:  [' До свидания!']
Result emotion:  joy angry
[' кучу денег для того, чтобы притащиться в эту дару.']
Time:  0.8496663570404053
Result text:  [' До свидания!']
Result emotion:  joy angry
[' Для того, что притащится в эту дару, это что вообще']
Time:  1.6858799457550049
Result text:  [' До свидания!']
Result emotion:  joy angry
[' что притащится в эту дару. Это что вообще такое? Посмотри на фето.']
Time:  2.4984517097473145
Result text:  [' До свидания!']
Result emotion:  joy angry
[' В эту дару это что вообще такое посовинайся центов?']
Time:  3.3959803581237793
Result text:  [' До свидания!']
Result emotion:  joy angry
[' Это что вообще такое? Посмотри на офицантов. Они все в черных какими-то']
Time:  4.181432247161865
Re

Метрика WER (ошибка по словам)

In [9]:
def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer

In [10]:
rez_text_all

[' Слушай, я потратил обретьом кучу денег для того, чтобы притоситься в эту дару. Это что вообще такое? Посмотри на официантов, они все в черных каких-то рубашках с кислыми минами. Даже никто из них до сих пор не подошел к нам.']

In [11]:
rez_all_result 


[[0.0,
  ' Слушай, потратил об этом кучу денег. Для того, чтобы предощиться.',
  'Слушай,',
  'neutral',
  'angry'],
 [0.5,
  ' кучу денег для того, чтобы притащиться в эту дару.',
  'кучу',
  'neutral',
  'angry'],
 [1.0,
  ' Для того, что притащится в эту дару, это что вообще',
  'того,',
  'neutral',
  'angry'],
 [1.5,
  ' что притащится в эту дару. Это что вообще такое? Посмотри на фето.',
  'что',
  'enthusiasm',
  'angry'],
 [2.0,
  ' В эту дару это что вообще такое посовинайся центов?',
  'эту',
  'enthusiasm',
  'angry'],
 [2.5,
  ' Это что вообще такое? Посмотри на офицантов. Они все в черных какими-то',
  'Это',
  'enthusiasm',
  'angry'],
 [3.0,
  ' это куба с Ринаэфи Цантов, они все в черных каких-то рубашках.',
  'с',
  'neutral',
  'angry'],
 [3.5,
  ' Центров, они все в черных каких-то рубашках с кислыми',
  'они',
  'neutral',
  'angry'],
 [4.0,
  ' Все в черных каких-то рубашках с кислыми минами.',
  'в',
  'anger',
  'angry'],
 [4.5, ' с кислыми минами.', 'с', 'neutra

In [18]:
s2 = s1[10:]
s3 = s1[:30]
ff = []
print(s3,s2)
for i in range(15):
    ff.append([s3[i:i+4], jellyfish.jaro_similarity(s3[i:], s2[:len(s3[i:])])])
ff    

NameError: name 's1' is not defined

In [25]:
import re
matches = re.finditer(' ', s1)
indices = [match.start() for match in matches]

indices

[0, 8, 17, 20, 25, 30, 37, 41, 47, 53]

In [26]:
import pandas as pd
df = pd.DataFrame(rez_all_result , columns= ['t', 'sub_string', 'word', 'em1', 'em2'])#.to_csv(path_a.split('/')[-1].split('.')[0] +  '.csv')
df

,t,sub_string,word,em1,em2
0,0.0,"Слушай, потратил об этом кучу денег. Для того...","Слушай,",neutral,angry
1,0.0,"кучу денег для того, чтобы притащиться в эту ...",кучу,neutral,angry
2,0.0,"Для того, что притащится в эту дару, это что ...",Для,neutral,angry
3,0.0,что притащится в эту дару. Это что вообще так...,что,enthusiasm,angry
4,0.0,В эту дару это что вообще такое посовинайся ц...,В,enthusiasm,angry
5,0.0,Это что вообще такое? Посмотри на офицантов. ...,Это,enthusiasm,angry
6,0.0,"это куба с Ринаэфи Цантов, они все в черных к...",это,neutral,angry
7,0.0,"Центров, они все в черных каких-то рубашках с...","Центров,",neutral,angry
8,0.0,Все в черных каких-то рубашках с кислыми минами.,Все,anger,angry
9,0.0,с кислыми минами.,с,neutral,angry


In [27]:
df.to_csv('traekt01.csv')

In [31]:
df['k2'] = 0
df['k1'] = 0

def concat_drop(df, plot_rez = 1):
    k0 = 0
    s1 = df.sub_string.values[0].lower()
    l1 = s1.split(' ')
    l1f = [s for s in l1 if len(s)>3 ]
    l1fi = [1 if len(s)>4 else 0 for s in l1]
    s = s1
    matches = re.finditer(' ', s1)
    indices1 = [match.start() for match in matches]
    for ik, s2 in enumerate(df.sub_string.values[1:].tolist()):
        # print(s1,s2)
        s2 = s2.lower()
        l2 = s2.split(' ')
        l2f = [s for s in l2 if len(s)>3 ]
        l2fi = [1 if len(s)>4 else 0 for s in l2]
        list_compare = []
        matches = re.finditer(' ', s2)
        indices2 = [match.start() for match in matches]
        
        for word in l2f:
            jaro_score = lambda x: jellyfish.jaro_similarity(x, word)
            list_compare += [list(map(jaro_score, l1f))]
        # print(list_compare)    
        k = np.argmax(list_compare)
        m = np.max(list_compare)
        if m>0.8:
            i, j = k%len(l1f), k//len(l1f)
                
             
            k1 = s1.find(l1f[i])
            k2 = s2.find(l2f[j]) 
            df['k1'].values[ik]=k1
            df['k2'].values[ik]=k2
            
            # k1 = k1 + len(s1[:k1].split(' ')[-1])
            if k2>=0:
                # print(':',[s1, ' ' * k1 + s2[k2:]])
                s = s[:k0+k1-1]+s2[k2:]
                k0 = s.find(s2[k2:]) 
                if plot_rez:
                    print(l1f[i],l2f[j]) 
                    print(s1,s2)
                    print(k0, k1, k2,': ', s[:k0]+'|'+s[k0:])
        s1 = s2
        l1 = l2.copy()
        l1f = l2f.copy()
        l1fi = l2fi.copy()
    return s, df   

concat_drop(df, plot_rez=0)

(' слушай, потратил об этомкучу денег для того, что притпритащится в эту дару. это что вообвообще такое посовивообще такое? посмотри на офицантов. они все в черных кчерных каких-то рубашках.черных каких-то рубчерных каких-то рубашках с кислымкикислыми минами. даже ниниктникто из них до сих пор не подошподошел к нам.',
       t                                         sub_string      word  \
 0   0.0   Слушай, потратил об этом кучу денег. Для того...   Слушай,   
 1   0.0   кучу денег для того, чтобы притащиться в эту ...      кучу   
 2   0.0   Для того, что притащится в эту дару, это что ...       Для   
 3   0.0   что притащится в эту дару. Это что вообще так...       что   
 4   0.0   В эту дару это что вообще такое посовинайся ц...         В   
 5   0.0   Это что вообще такое? Посмотри на офицантов. ...       Это   
 6   0.0   это куба с Ринаэфи Цантов, они все в черных к...       это   
 7   0.0   Центров, они все в черных каких-то рубашках с...  Центров,   
 8   0.0   Все в черных

In [ ]:
df

# Поиск ключевого слова в речи
https://huggingface.co/learn/audio-course/ru/chapter7/voice-assistant


Слушаем объект микрофон и его записываем в текст

In [29]:
from transformers.pipelines.audio_utils import ffmpeg_microphone_live
import time
chunk_length_s=2.0
stream_chunk_s=0.25
sampling_rate =  16000
mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )
# model_text.run(next(mic)['raw'], sr)

In [21]:
next(mic)

{'raw': array([-0.00887134, -0.01059099, -0.01062613, ...,  0.31721792,
         0.3201145 ,  0.31525022], dtype=float32),
 'stride': (5333, 5333),
 'partial': False,
 'sampling_rate': 16000}

In [44]:
mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )
for _ in range(20):
    t1 = time.time()
    prediction = model_text.run(next(mic)['raw'])
    print(time.time()-t1, prediction)
    

0.7853002548217773 [' Редактор субтитров А.Синецкая Корректор А.Сухиашвили']
0.5997827053070068 [' Редактор субтитров А.Синецкая Корректор А.Сухиашвили']
0.38577866554260254 [' Дай']
0.4005601406097412 [' То есть есть...']
0.3913116455078125 [' То есть, если мы']
0.4379909038543701 [' То есть, если мы говорим']
0.45891761779785156 [' То есть, если мы говорим,']
0.4319002628326416 [' То есть, если мы говорим,']
0.4264817237854004 [' Мари, мода!']
0.40451836585998535 [' Марин, Мана!']
0.4573493003845215 [' Молода, в тюрьме.']
0.4205305576324463 [' Марин, мана в теории.']
0.4138205051422119 [' В тюрье должна быть']
0.4211001396179199 [' Добрый день, Должна.']
4.267004013061523 [' И вот это, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что я не знаю, что

In [ ]:
# https://huggingface.co/learn/audio-course/ru/chapter7/voice-assistant

# Использование микрофона

In [24]:
from transformers.pipelines.audio_utils import ffmpeg_microphone_live
from transformers import pipeline
import time
chunk_length_s=1.0
stream_chunk_s=0.25
sampling_rate =  16000

import sys
device = 'cpu'
transcriber = pipeline(
    "automatic-speech-recognition", model="Shirali/whisper-small-ru", device=device
)
sampling_rate = transcriber.feature_extractor.sampling_rate

mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

def transcribe(chunk_length_s=chunk_length_s, stream_chunk_s=stream_chunk_s, mic=mic, sampling_rate = sampling_rate ):
    rez = []
    print("Start speaking...")
    for item in transcriber(mic, generate_kwargs={"max_new_tokens": 64}):
        sys.stdout.write("\033[K")
        # print(item["text"], end="\r")
        # rez.append(item["text"])
        if not item["partial"][0]:
            rez.append(item["text"])
            print(item["text"])
            break

    return rez

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
# t1 = time.time()
for _ in range(3):
    t1 = time.time()
    print(transcribe())
    print(time.time()-t1)    

Start speaking...
– привет, привет
['– привет, привет']
7.168356418609619
Start speaking...
а как ты
['а как ты']
5.515061378479004
Start speaking...

['']
19.813069581985474


In [7]:
# !pip install pyspellchecker

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 10.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [43]:
from spellchecker import SpellChecker # pip install pyspellchecker
spell = SpellChecker(language='ru')
for word in 'как тебе это положено распространовать. Давай посмотрим на то то'.split(' '):
    # word  # "слово для проверки орфографии
    if spell.correction(word) == word:
        print(word, " : Слово написано верно!")
    else:
        print("Ошибка в написании слова:", word)
        print("Возможные варианты:")
        print(spell.candidates(word))

как  : Слово написано верно!
тебе  : Слово написано верно!
это  : Слово написано верно!
положено  : Слово написано верно!
Ошибка в написании слова: распространовать.
Возможные варианты:
None
Давай  : Слово написано верно!
посмотрим  : Слово написано верно!
на  : Слово написано верно!
то  : Слово написано верно!
то  : Слово написано верно!


# Конвертация моделей из Hugginface

In [6]:
import glob

path_mix_dn = '../шумоподавление/DTLN_output/*.wav'
path_mix_n = '../шумоподавление/Noisereduce_output/*.wav'

In [7]:
list_mix_dn = glob.glob(path_mix_dn)
len(list_mix_dn)
list_mix_dn

['../шумоподавление/DTLN_output/DTLN_39_sadness_anger_s_020.wav',
 '../шумоподавление/DTLN_output/DTLN_49_sadness_happiness_h_080.wav',
 '../шумоподавление/DTLN_output/DTLN_02_anger_sadness a_051.wav',
 '../шумоподавление/DTLN_output/DTLN_33_sadness_disgust_d_020.wav',
 '../шумоподавление/DTLN_output/DTLN_07_fear_neutral f_050.wav',
 '../шумоподавление/DTLN_output/DTLN_05_neutral_fear n_020.wav',
 '../шумоподавление/DTLN_output/DTLN_48_fear_disgust_d_070.wav',
 '../шумоподавление/DTLN_output/DTLN_21_happiness_anger_a_110.wav',
 '../шумоподавление/DTLN_output/DTLN_05_neutral_fear f_061.wav',
 '../шумоподавление/DTLN_output/DTLN_18_happiness_neutral_h_080.wav',
 '../шумоподавление/DTLN_output/DTLN_20_disgust_enthusiasm_d_030.wav',
 '../шумоподавление/DTLN_output/DTLN_43_enthusiasm_neutral_n_110.wav',
 '../шумоподавление/DTLN_output/DTLN_42_anger_fear_f_030.wav',
 '../шумоподавление/DTLN_output/DTLN_22_fear_anger_a_100.wav',
 '../шумоподавление/DTLN_output/DTLN_32_happiness_enthusiasm_h_0

In [8]:
list_mix_n = glob.glob(path_mix_n)
len(list_mix_n)

783

In [9]:
list_mix_dn[:6], list_mix_n[:6], 

(['../шумоподавление/DTLN_output/DTLN_39_sadness_anger_s_020.wav',
  '../шумоподавление/DTLN_output/DTLN_49_sadness_happiness_h_080.wav',
  '../шумоподавление/DTLN_output/DTLN_02_anger_sadness a_051.wav',
  '../шумоподавление/DTLN_output/DTLN_33_sadness_disgust_d_020.wav',
  '../шумоподавление/DTLN_output/DTLN_07_fear_neutral f_050.wav',
  '../шумоподавление/DTLN_output/DTLN_05_neutral_fear n_020.wav'],
 ['../шумоподавление/Noisereduce_output/Noisereduce_27_neutral_fear_n_071.wav',
  '../шумоподавление/Noisereduce_output/Noisereduce_03_disgust_neutral d_040.wav',
  '../шумоподавление/Noisereduce_output/Noisereduce_05_neutral_fear f_070.wav',
  '../шумоподавление/Noisereduce_output/Noisereduce_01_happiness_anger h_100.wav',
  '../шумоподавление/Noisereduce_output/Noisereduce_a5039325b3cf00b727438744ded26b74.wav',
  '../шумоподавление/Noisereduce_output/Noisereduce_06_happiness_enthusiasm h_072.wav'])

In [10]:
import pandas as pd

In [38]:
df_text = pd.read_csv('../dataset/RESD_csv/train.csv')
df_text.head()

,name,path,emotion,text
0,32_happiness_enthusiasm_h_120,happiness_enthusiasm_32/32_happiness_enthusias...,happiness,"Конечно, расскажу, обязательно. Ой, сейчас рас..."
1,36_disgust_happiness_d_130,disgust_happiness_36/36_disgust_happiness_d_13...,disgust,Вы ещё и профессию решили поменять.
2,34_anger_fear_a_060,anger_fear_34/34_anger_fear_a_060.wav,anger,"Ты знаешь, чем это для тебя закончится?"
3,25_anger_disgust_a_010,anger_disgust_25/25_anger_disgust_a_010.wav,anger,Добрый день. Вы хотели бы приобрести недвижимо...
4,17_neutral_disgust_d_092,neutral_disgust_17/17_neutral_disgust_d_092.wav,disgust,"все ваши рекламные акции, пожалуйста, больше н..."


In [34]:
df_text.columns

Index(['name', 'path', 'emotion', 'text'], dtype='object')

In [11]:
mix_project_path = '../TZ_noisa/'
mix_file = 'mix.csv'
fd = pd.read_csv(mix_project_path + mix_file)

In [12]:
fd.head()

,audio_path,noise_path,mix_path,bitrate,duration,mix_method,volume_sound,volume_noise
0,train/enthusiasm_neutral_43/43_enthusiasm_neut...,noise/-391904074655300970.wav,mix_test/43_enthusiasm_neutral_e_080.wav,16000,00:04.6,1,8.428574,1.811726
1,test/anger_fear_42/42_anger_fear_f_070.wav,noise/1248824479100550917.wav,mix_test/42_anger_fear_f_070.wav,16000,00:04.3,1,0.214053,6.887027
2,train/disgust_happiness_36/36_disgust_happines...,noise/-392355194681447233.wav,mix_test/36_disgust_happiness_d_100.wav,16000,00:02.2,1,9.884556,9.882694
3,train/enthusiasm_sadness_15/15_enthusiasm_sadn...,noise/1147034587767574128.wav,mix_test/15_enthusiasm_sadness_s_031.wav,16000,00:07.3,1,3.965600,7.601287
4,train/fear_disgust_48/48_fear_disgust_d_100.wav,noise/1279042682565365761.wav,mix_test/48_fear_disgust_d_100.wav,16000,00:06.7,1,2.940736,1.462580


In [50]:
path_buf =  '../dataset/RESD_csv/'
list_rez = []
list_problem = []

model_text = SpeechtoText('whisper-tiny_onnx', "openai/whisper-base", "openai/whisper-tiny")
for name in list_mix_dn[:100]:
    try:
        name_file = name.split('/')[-1]
        name_n = '../шумоподавление/Noisereduce_output/Noisereduce_' + '_'.join(name_file.split('_')[1:])
    
        name_mix = 'mix_test/' + '_'.join(name_file.split('_')[1:])
        
        name_ideal = fd.loc[fd.mix_path == name_mix, :].values
        
        trek_name = name_ideal[0][0].split('/')[-1].split('.')[0]
        # print(name_ideal, trek_name )
        name_em_ideal = df_text.loc[df_text.iloc[:,0] == trek_name, ['emotion', 	'text']].values[0]
        em_ideal, text_ideal = name_em_ideal
        
        # print(name, name_n, name_ideal)
        path_a1 =path_buf + name_ideal[0][0]
        t1 = time.time()
        speech, sr = librosa.load(path_a1, sr=sampling_rate)
    
        rez_text = model_text.run(speech)
    
        path_a2 = name_n
        t1 = time.time()
        speech, sr = librosa.load(path_a2, sr=sampling_rate)
        
        rez_text_n = model_text.run(speech) 
    
        path_a3 = name
        t1 = time.time()
        speech, sr = librosa.load(path_a3, sr=sampling_rate)
        
        rez_text_dn = model_text.run(speech)
        # print(path_a1, rez_text, '\n',path_a3, rez_text_dn[0], '\n', path_a2, rez_text_n[0])
        list_rez.append([path_a1,path_a3, path_a2, rez_text[0], rez_text_dn[0], rez_text_n[0], text_ideal, em_ideal ])
        # print(rez_text)
    except:
        print(name)
        list_problem.append(name)
        

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


../dataset/RESD_csv/train/sadness_anger_39/39_sadness_anger_s_020.wav [' Но он уже не дышит, а все хорошо, что мне делать.'] 
 ../шумоподавление/DTLN_output/DTLN_39_sadness_anger_s_020.wav  Умором Лунджене веточный сык, что у него не втикаст. 
 ../шумоподавление/Noisereduce_output/Noisereduce_39_sadness_anger_s_020.wav  Он уже уже к суху черненький.
../dataset/RESD_csv/train/sadness_happiness_49/49_sadness_happiness_h_080.wav [' Я сейчас помню и отдам тебе. Если тебе что-то не устраивает, не мои проблемы.'] 
 ../шумоподавление/DTLN_output/DTLN_49_sadness_happiness_h_080.wav  Я сейчас помню и отдам тебе. Если тебе что-то не устраивает, не мои проблемы. 
 ../шумоподавление/Noisereduce_output/Noisereduce_49_sadness_happiness_h_080.wav  Я сейчас помню, я отдам тебе, если тебе что-то не устраивает. У моей проблемы.
../dataset/RESD_csv/train/02_anger_sadness/02_anger_sadness a_051.wav [' Неужели? Дольше. А на Ивану она 70 лет. Она делает быстрее, чем ты. А ты молодой специалист, конечно.'] 


In [46]:
pd.DataFrame(list_rez, columns=['name', 'name2', 'name2', 'ideal', 'dn', 'n', 'text', 'em', ]).to_csv('rez_test_noise.csv')

In [47]:
res = pd.DataFrame(list_rez, columns=['name', 'name2', 'name2', 'ideal', 'dn', 'n', 'text', 'em',])

In [48]:
res

,name,name2,name2,ideal,dn,n,text,em
0,../dataset/RESD_csv/train/sadness_anger_39/39_...,../шумоподавление/DTLN_output/DTLN_39_sadness_...,../шумоподавление/Noisereduce_output/Noiseredu...,"Но он уже не дышит, а все хорошо, что мне дел...","Умором Лунджене веточный сык, что у него не в...",Он уже уже к суху черненький.,Он уже не дышит. Что мне делать?,sadness
1,../dataset/RESD_csv/train/sadness_happiness_49...,../шумоподавление/DTLN_output/DTLN_49_sadness_...,../шумоподавление/Noisereduce_output/Noiseredu...,Я сейчас помню и отдам тебе. Если тебе что-то...,Я сейчас помню и отдам тебе. Если тебе что-то...,"Я сейчас помню, я отдам тебе, если тебе что-т...","Я сейчас помою и отдам тебе, если тебя что-то ...",happiness
2,../dataset/RESD_csv/train/02_anger_sadness/02_...,../шумоподавление/DTLN_output/DTLN_02_anger_sa...,../шумоподавление/Noisereduce_output/Noiseredu...,Неужели? Дольше. А на Ивану она 70 лет. Она д...,Неужели дольше? А на Иваном 70 лет. Он делает...,Неужели дольше? Он на Иваном 70 лет. Он делае...,Неужели дольше? Анна Ивановна 70 лет. Она дела...,anger
3,../dataset/RESD_csv/train/sadness_disgust_33/3...,../шумоподавление/DTLN_output/DTLN_33_sadness_...,../шумоподавление/Noisereduce_output/Noiseredu...,Мало ли что-то захотел.,"Пока и решили, чтобы западаться его.","Смотри, что ты так красиво.",Мало ли что ты захотел,disgust


In [26]:
n = 10
res.iloc[n]

name     ../dataset/RESD_csv/test/disgust_enthusiasm_20...
name2    ../шумоподавление/DTLN_output/DTLN_20_disgust_...
name2    ../шумоподавление/Noisereduce_output/Noiseredu...
ideal                      Жена в олере. Твоего любовника.
dn                                            Жена валина.
n                                          Зима в алиэрия!
Name: 10, dtype: object

In [27]:
res.iloc[n]['dn']

' Жена валина.'

In [28]:
res.iloc[n]['name2'].to_list()

['../шумоподавление/DTLN_output/DTLN_20_disgust_enthusiasm_d_030.wav',
 '../шумоподавление/Noisereduce_output/Noisereduce_20_disgust_enthusiasm_d_030.wav']

In [29]:
res.iloc[n]['name']

'../dataset/RESD_csv/test/disgust_enthusiasm_20/20_disgust_enthusiasm_d_030.wav'